# Exploring sea ice freeboards with ICESat-2 (ATL10)

Information obtained primarily from the ATL07/10 Algorithm Theoretical Basis Document (ATBD, Kwok et al., 2019)     and the NSIDC product description page: https://nsidc.org/data/atl10.   

Notebook author: Alek Petty, relying extensively on above product manuals. 
Description: Notebook describing the ICESat-2 ATL10 product.   
Input requirements: Demo ATL10 data file   
Date: June 2019
More info: See the ATL07/ATL10 Algorithm Theoretical Basis Document (ATBD): https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/ICESat2_ATL07_ATL10_ATBD_r001.pdf   


## Notebook objectives
* General understanding of what's included in a typical ATL07 file.
* Plotting and basic analysis of ATL07 data.
* Potentially some info on reading in and analyzing a large quantity of ATL07 data!


## Notebook instructions
1. Follow along with the notebook tutorial. 
2. Play around changing options and re-running the relevant notebook cells. 

Here I use the hdf5 file from: https://nsidc.org/data/atl07   
For the demo below I'm using the file: X
If using this using the ICESAT-2 Pangeo instance, you can download the file using...


### Notes      
* Check out the known issues document:https://nsidc.org/sites/nsidc.org/files/technical-references/ATL0710-KnownIssues.pdf



In [ ]:
#Magic function to enable interactive plotting in Jupyter notebook
#Allows you to zoom/pan within plots after generating
#Normally, this would be %matplotlib notebook, but since we're using Juptyerlab, we need a different widget
#%matplotlib notebook
%matplotlib inline

In [ ]:
#Import necesary modules
#Use shorter names (np, pd, plt) instead of full (numpy, pandas, matplotlib.pylot) for convenience
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import convert_GPS_time as cGPS
import seaborn as sns
import pandas as pd
import h5py  

In [ ]:
# Use seasborn for nicer looking inline plots
sns.set(context='notebook', style='darkgrid')
#st = axes_style("whitegrid")

file_path = './Data/'
ATL10_filename = X
ATL10_file_path = file_path + ATL10_filename
beamStr='gt1r'

In [ ]:
def getATL10FreeboardShotData(fileT, mapProj, beamStr='gt1r'):
    """
    Load ATL10 freeboard shot data from a given beam
    # Store in a Pandas dataframe

    Args:
        freeboardFileT (file): file path of ICESat freeboard data
        mapProj (basemap instance): basemap map projection
        beamStr (str): ground track (gt1, gt2, gt3) then the weak (l) and strong (r) beam
            i.e. gt1r is the strong beam from ground track 1

    
    Returns:
        dF (var): Dataframe containing freeboard, year, month, day, lon, lat, x, y

    I think the dates are indexed starting from 1 - i.e. month of 1 = January 
        
    """

    print('ATL10 file:', fileT)
    
    f1 = h5py.File(fileT, 'r')
    try:
        freeboard=f1[beamStr]['freeboard_beam_segment']['beam_freeboard']['beam_fb_height'][:]
    except:
        return 'No good shot data'

    freeboard_confidence=f1[beamStr]['freeboard_beam_segment']['beam_freeboard']['beam_fb_confidence'][:]
    freeboard_quality=f1[beamStr]['freeboard_beam_segment']['beam_freeboard']['beam_fb_quality_flag'][:]
    
    lons=f1[beamStr]['freeboard_beam_segment']['beam_freeboard']['longitude'][:]
    lats=f1[beamStr]['freeboard_beam_segment']['beam_freeboard']['latitude'][:]
    deltaTime=f1[beamStr]['freeboard_beam_segment']['beam_freeboard']['delta_time'][:]

    dF = pd.DataFrame({'freeboard':freeboard, 'lon':lons, 'lat':lats, 'delta_time':deltaTime})

    dF = dF[(dF['freeboard']>0)]
    dF = dF[(dF['freeboard']<10)]

    # Decide here if we want to also filter based on the confidence and/or quality flag

    # Reset row indexing
    dF=dF.reset_index(drop=True)

    #print(fileT)
    #fileStr=fileT.split("/ATL10_")[-1]
    #print(fileStr)
    #dF['year'] = int(fileStr[0:4])
    #dF['month'] = int(fileStr[4:6])
    #dF['day'] = int(fileStr[6:8])

    print('Year:', fileStr[0:4], 'Month:', fileStr[4:6], 'Day:', fileStr[6:8])
    

    #xpts, ypts=mapProj(dF['lon'].values, dF['lat'].values)

    #dF['xpts'] = pd.Series(xpts, index=dF.index)
    #dF['ypts'] = pd.Series(ypts, index=dF.index)

    #dF['shot'] = beamStr
    

    return dF
